In [116]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
from haversine import haversine

In [52]:
areal = pd.read_csv('areal.csv', encoding='cp949')
areal = areal.iloc[1:,:]
areal.head()

,시도,시군구,레벨01,항목,단위,2018 년,2019 년,2020 년,2021 년,Unnamed: 9
1,서울,서울,계,면적,NaN,605238966.0,605237001.6,605228540.7,605243961.2,NaN
2,서울,종로구,계,면적,NaN,23913317.6,23913280.4,23913168.3,23912935.7,NaN
3,서울,중구,계,면적,NaN,9959837.2,9959982.5,9960304.5,9960262.0,NaN
4,서울,용산구,계,면적,NaN,21866257.4,21866383.5,21866144.1,21866145.2,NaN
5,서울,성동구,계,면적,NaN,16859545.5,16859342.7,16861003.9,16859898.9,NaN


In [53]:
areal = areal[['시도', '시군구', '2021 년']]
areal.columns = ['시도', '시군구', '면적']
areal.head()

,시도,시군구,면적
1,서울,서울,605243961.2
2,서울,종로구,23912935.7
3,서울,중구,9960262.0
4,서울,용산구,21866145.2
5,서울,성동구,16859898.9


In [54]:
areal = areal[ areal['시도'] != areal['시군구'] ]
areal.head()

,시도,시군구,면적
2,서울,종로구,23912935.7
3,서울,중구,9960262.0
4,서울,용산구,21866145.2
5,서울,성동구,16859898.9
6,서울,광진구,17062995.2


In [55]:
areal['주소'] = areal['시도'] + ' ' + areal['시군구']
areal.head()

,시도,시군구,면적,주소
2,서울,종로구,23912935.7,서울 종로구
3,서울,중구,9960262.0,서울 중구
4,서울,용산구,21866145.2,서울 용산구
5,서울,성동구,16859898.9,서울 성동구
6,서울,광진구,17062995.2,서울 광진구


In [45]:
client_id = 'client_id';    
client_pw = 'client_pw';  

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []     
for add in areal['주소']:
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)  
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])

geo_coordi
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"주소": areal['주소'].values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1]})

pd_geo_coordi.head()

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
'result' not exist!
Success!
Success!
Success!
Success!
'result' not exist!
Success!
Success!
Success!
Success!
'result' not exist!
Success!
Success!
Success!
Success!
Success!
Success!
'result' not exist!
Success!
Success!
'result' not exist!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!


,주소,위도,경도
0,서울 종로구,37.5735207,126.9788345
1,서울 중구,37.563843,126.997602
2,서울 용산구,37.532527,126.99049
3,서울 성동구,37.563456,127.036821
4,서울 광진구,37.538617,127.082375


In [58]:
areal_coordinate = pd.merge(areal, pd_geo_coordi, on= '주소')
areal_coordinate.head()

,시도,시군구,면적,주소,위도,경도
0,서울,종로구,23912935.7,서울 종로구,37.5735207,126.9788345
1,서울,중구,9960262.0,서울 중구,37.563843,126.997602
2,서울,용산구,21866145.2,서울 용산구,37.532527,126.99049
3,서울,성동구,16859898.9,서울 성동구,37.563456,127.036821
4,서울,광진구,17062995.2,서울 광진구,37.538617,127.082375


In [61]:
areal_coordinate = areal_coordinate.dropna()
areal_coordinate.to_csv('areal_coordinate.csv', encoding='utf-8-sig')

In [47]:
areal.to_csv('areal_coordinate.csv', encoding='utf-8-sig')

In [48]:
areal[areal['위도'].isnull()]

,시도,시군구,면적,주소,위도,경도


In [38]:
pd_geo_coordi = pd_geo_coordi.dropna()
pd_geo_coordi.head()

,주소,위도,경도
0,서울 종로구,37.5735207,126.9788345
1,서울 중구,37.563843,126.997602
2,서울 용산구,37.532527,126.99049
3,서울 성동구,37.563456,127.036821
4,서울 광진구,37.538617,127.082375


In [63]:
pop = pd.read_csv('pop.csv', encoding='cp949')
pop.head()

,행정구역(시군구)별,2017.01,2017.07,2018.01,2018.07,2019.01,2019.07,2020.01,2020.07,2021.01,2021.07,2022.01,2022.06
0,서울특별시,9930478,9908612,9851767,9806538,9766288,9751415,9733509,9715429,9657969,9558153,9505926,9494807
1,종로구,152405,154802,154619,153531,152866,151858,151215,150183,149125,145692,144543,143624
2,중구,124976,123926,125786,125941,125872,126414,126201,125970,125038,122951,122315,122088
3,용산구,229997,228766,229541,229545,229167,229155,229385,229432,230045,225977,222573,222263
4,성동구,301040,304236,304955,308519,307723,304292,300410,296896,293103,288703,285516,283821


In [70]:
capital = ['서울특별시', '부산광역시','대구광역시','인천광역시','광주광역시','대전광역시','울산광역시','세종특별자치시',
          '경기도','강원도','충청북도','충청남도','전라북도','전라남도','경상북도','경상남도','제주특별자치도']
capital_idx = np.where(pop['행정구역(시군구)별'].isin(capital))
capital_idx

(array([  0,  26,  43,  52,  63,  69,  75,  81,  83, 132, 151, 167, 185,
        202, 225, 251, 275], dtype=int64),)

In [69]:
pop['구역'] = None
pop.head()

,행정구역(시군구)별,2017.01,2017.07,2018.01,2018.07,2019.01,2019.07,2020.01,2020.07,2021.01,2021.07,2022.01,2022.06,구역
0,서울특별시,9930478,9908612,9851767,9806538,9766288,9751415,9733509,9715429,9657969,9558153,9505926,9494807,None
1,종로구,152405,154802,154619,153531,152866,151858,151215,150183,149125,145692,144543,143624,None
2,중구,124976,123926,125786,125941,125872,126414,126201,125970,125038,122951,122315,122088,None
3,용산구,229997,228766,229541,229545,229167,229155,229385,229432,230045,225977,222573,222263,None
4,성동구,301040,304236,304955,308519,307723,304292,300410,296896,293103,288703,285516,283821,None


In [ ]:
n = len(capital_idx[0]) -1
for i in range(n):
    if i==n:
        pop.loc[ capital_idx[0][i] : ,'구역'] = capital[i]
    else:
        pop.loc[ capital_idx[0][i] : (capital_idx[0][i+1]-1) ,'구역'] = capital[i]

In [92]:
pop.loc[ 275 : 277 ,'구역'] = '제주특별자치도'
pop = pop[ pop['행정구역(시군구)별'] != pop['구역'] ]
pop.head()

,행정구역(시군구)별,2017.01,2017.07,2018.01,2018.07,2019.01,2019.07,2020.01,2020.07,2021.01,2021.07,2022.01,2022.06,구역
1,종로구,152405,154802,154619,153531,152866,151858,151215,150183,149125,145692,144543,143624,서울특별시
2,중구,124976,123926,125786,125941,125872,126414,126201,125970,125038,122951,122315,122088,서울특별시
3,용산구,229997,228766,229541,229545,229167,229155,229385,229432,230045,225977,222573,222263,서울특별시
4,성동구,301040,304236,304955,308519,307723,304292,300410,296896,293103,288703,285516,283821,서울특별시
5,광진구,357303,357419,357785,355352,355382,353206,351263,349258,346067,342111,340002,339893,서울특별시


In [93]:
pop['주소'] = pop['구역'] + ' ' + pop['행정구역(시군구)별']
pop.head()

C:\Users\INHA\AppData\Local\Temp/ipykernel_3328/1742732902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop['주소'] = pop['구역'] + ' ' + pop['행정구역(시군구)별']


,행정구역(시군구)별,2017.01,2017.07,2018.01,2018.07,2019.01,2019.07,2020.01,2020.07,2021.01,2021.07,2022.01,2022.06,구역,주소
1,종로구,152405,154802,154619,153531,152866,151858,151215,150183,149125,145692,144543,143624,서울특별시,서울특별시 종로구
2,중구,124976,123926,125786,125941,125872,126414,126201,125970,125038,122951,122315,122088,서울특별시,서울특별시 중구
3,용산구,229997,228766,229541,229545,229167,229155,229385,229432,230045,225977,222573,222263,서울특별시,서울특별시 용산구
4,성동구,301040,304236,304955,308519,307723,304292,300410,296896,293103,288703,285516,283821,서울특별시,서울특별시 성동구
5,광진구,357303,357419,357785,355352,355382,353206,351263,349258,346067,342111,340002,339893,서울특별시,서울특별시 광진구


In [107]:
pop.iloc[ np.where(pop['행정구역(시군구)별']=='세종시')[0] , 14] = '세종시'

C:\Users\INHA\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [108]:
pop.iloc[ np.where(pop['행정구역(시군구)별']=='세종시')[0] ]

,행정구역(시군구)별,2017.01,2017.07,2018.01,2018.07,2019.01,2019.07,2020.01,2020.07,2021.01,2021.07,2022.01,2022.06,구역,주소
82,세종시,244939,266075,283198,302703,316814,330298,342328,346217,357871,364364,374377,380215,세종특별자치시,세종시


In [111]:
api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []     
for add in pop['주소']:
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)  
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])

geo_coordi
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"주소": pop['주소'].values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1]})

pd_geo_coordi.head()

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
S

,주소,위도,경도
0,서울특별시 종로구,37.5735207,126.9788345
1,서울특별시 중구,37.563843,126.997602
2,서울특별시 용산구,37.532527,126.99049
3,서울특별시 성동구,37.563456,127.036821
4,서울특별시 광진구,37.538617,127.082375


In [112]:
pop_coordi = pd.merge(pop, pd_geo_coordi, on= '주소')
pop_coordi.head()

,행정구역(시군구)별,2017.01,2017.07,2018.01,2018.07,2019.01,2019.07,2020.01,2020.07,2021.01,2021.07,2022.01,2022.06,구역,주소,위도,경도
0,종로구,152405,154802,154619,153531,152866,151858,151215,150183,149125,145692,144543,143624,서울특별시,서울특별시 종로구,37.5735207,126.9788345
1,중구,124976,123926,125786,125941,125872,126414,126201,125970,125038,122951,122315,122088,서울특별시,서울특별시 중구,37.563843,126.997602
2,용산구,229997,228766,229541,229545,229167,229155,229385,229432,230045,225977,222573,222263,서울특별시,서울특별시 용산구,37.532527,126.99049
3,성동구,301040,304236,304955,308519,307723,304292,300410,296896,293103,288703,285516,283821,서울특별시,서울특별시 성동구,37.563456,127.036821
4,광진구,357303,357419,357785,355352,355382,353206,351263,349258,346067,342111,340002,339893,서울특별시,서울특별시 광진구,37.538617,127.082375


In [113]:
region_info = pd.merge(pop_coordi, areal_coordinate, on= '위도',how = 'outer')

In [115]:
region_info.to_csv('region_info.csv', encoding='utf-8-sig')

In [118]:
pop_coordi.shape[0]

261

In [137]:
ori = []
des = []
dist = []
n = pop_coordi.shape[0]
for i in range(n):
    for j in range(i,n):
        ori.append(pop_coordi.iloc[i,14])
        des.append(pop_coordi.iloc[j,14])
        tem_dist = haversine( pop_coordi.iloc[i,15:17].values.astype(float), pop_coordi.iloc[j,15:17].values.astype(float))
        dist.append(tem_dist)

In [148]:
region_dist = pd.DataFrame({"지역1": ori,
                              "지역2": des,
                              "거리": dist})

In [150]:
region_dist = region_dist.loc[np.where(region_dist['거리']!=0)[0],]
region_dist.to_csv('region_dist.csv', encoding='utf-8-sig')